# 1   Natural Language Understanding

## 1.1   Querying a Database

In [1]:
import nltk
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


In [2]:
from nltk import load_parser
cp = load_parser('grammars/book_grammars/sql0.fcfg', trace=3)
query = 'What cities are located in China'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q)

|.W.c.a.l.i.C.|
Leaf Init Rule:
|[-] . . . . .| [0:1] 'What'
|. [-] . . . .| [1:2] 'cities'
|. . [-] . . .| [2:3] 'are'
|. . . [-] . .| [3:4] 'located'
|. . . . [-] .| [4:5] 'in'
|. . . . . [-]| [5:6] 'China'
Feature Bottom Up Predict Combine Rule:
|[-] . . . . .| [0:1] Det[SEM='SELECT'] -> 'What' *
Feature Bottom Up Predict Combine Rule:
|[-> . . . . .| [0:1] NP[SEM=(?det+?n)] -> Det[SEM=?det] * N[SEM=?n] {?det: 'SELECT'}
Feature Bottom Up Predict Combine Rule:
|. [-] . . . .| [1:2] N[SEM='City FROM city_table'] -> 'cities' *
Feature Single Edge Fundamental Rule:
|[---] . . . .| [0:2] NP[SEM=(SELECT, City FROM city_table)] -> Det[SEM='SELECT'] N[SEM='City FROM city_table'] *
Feature Bottom Up Predict Combine Rule:
|[---> . . . .| [0:2] S[SEM=(?np+WHERE+?vp)] -> NP[SEM=?np] * VP[SEM=?vp] {?np: (SELECT, City FROM city_table)}
Feature Bottom Up Predict Combine Rule:
|. . [-] . . .| [2:3] IV[SEM=''] -> 'are' *
Feature Bottom Up Predict Combine Rule:
|. . [-> . . .| [2:3] VP[SEM=(?v+?pp)] 

In [3]:
from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for r in rows: print(r[0], end=" ")

canton chungking dairen harbin kowloon mukden peking shanghai sian tientsin 

In [4]:
nltk.data.show_cfg('grammars/book_grammars/sql1.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
VP[SEM=(?v + ?np)] -> TV[SEM=?v] NP[SEM=?np]
VP[SEM=(?vp1 + ?c + ?vp2)] -> VP[SEM=?vp1] Conj[SEM=?c] VP[SEM=?vp2]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
NP[SEM=(?n + ?pp)]  -> N[SEM=?n] PP[SEM=?pp]
NP[SEM=?n]  -> N[SEM=?n]  | CardN[SEM=?n] 
CardN[SEM='1000'] -> '1,000,000' 
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
Conj[SEM='AND'] -> 'and'
N[SEM='City FROM city_table'] -> 'cities'
N[SEM='Population'] -> 'populations'
IV[SEM=''] -> 'are'
TV[SEM=''] -> 'have'
A -> 'located'
P[SEM=''] -> 'in'
P[SEM='>'] -> 'above'


In [5]:
cp = load_parser('grammars/book_grammars/sql1.fcfg',)
query = 'What cities are in China and have populations above 1,000,000'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q)

SELECT City FROM city_table WHERE Country="china" AND Population > 1000


## 1.2   Natural Language, Semantics and Logic

# 2   Propositional Logic

In [6]:
nltk.boolean_ops()

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->


In [7]:
read_expr = nltk.sem.Expression.fromstring
read_expr('-(P & Q)')

<NegatedExpression -(P & Q)>

In [8]:
read_expr('P & Q')

<AndExpression (P & Q)>

In [9]:
read_expr('P | (R -> Q)')

<OrExpression (P | (R -> Q))>

In [10]:
read_expr('P <-> -- P')

<IffExpression (P <-> --P)>

In [11]:
lp = nltk.sem.Expression.fromstring
SnF = read_expr('SnF')
NotFnS = read_expr('-FnS')
R = read_expr('SnF -> -FnS')
prover = nltk.Prover9()
prover.prove(NotFnS, [SnF, R])

LookupError: 

===========================================================================
NLTK was unable to find the prover9 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on prover9, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [ ]:
val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])

In [ ]:
val['P']

In [ ]:
dom = set()
g = nltk.Assignment(dom)

In [ ]:
m = nltk.Model(dom, val)

In [ ]:
print(m.evaluate('(P & Q)', g))

In [ ]:
print(m.evaluate('-(P & Q)', g))

In [ ]:
print(m.evaluate('(P & R)', g))

In [ ]:
print(m.evaluate('(P | R)', g))

In [ ]:
print(m.evaluate('(P -> R)', g))

In [ ]:
print(m.evaluate('(P <-> R)', g))

In [ ]:
print(m.evaluate('(P | R) -> -(P & R)', g))

# 3   First-Order Logic

## 3.1   Syntax

In [ ]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr('walk(angus)', type_check=True)
expr.argument

In [ ]:
expr.argument.type

In [ ]:
expr.function

In [ ]:
expr.function.type

In [ ]:
sig = {'walk': '<e, t>'}
expr = read_expr('walk(angus)', signature=sig)
expr.function.type

In [ ]:
read_expr = nltk.sem.Expression.fromstring
read_expr('dog(cyril)').free()

In [ ]:
read_expr('dog(x)').free()

In [ ]:
read_expr('own(angus, cyril)').free()

In [ ]:
read_expr('exists x.dog(x)').free()

In [ ]:
read_expr('((some x. walk(x)) -> sing(x))').free()

In [ ]:
read_expr('exists x.own(y, x)').free()

## 3.2   First Order Theorem Proving

In [ ]:
NotFnS = read_expr('-north_of(f, s)')
SnF = read_expr('north_of(s, f)')
R = read_expr('all x. all y. (north_of(x, y) -> -north_of(y, x))')
prover = nltk.Prover9()
prover.prove(NotFnS, [SnF, R])

In [ ]:
FnS = read_expr('north_of(f, s)')
prover.prove(FnS, [SnF, R])

## 3.3   Summarizing the Language of First Order Logic

## 3.4   Truth in Model

In [ ]:
dom = {'b', 'o', 'c'}

In [ ]:
v = """
    bertie => b
    olive => o
    cyril => c
    boy => {b}
    girl => {o}
    dog => {c}
    walk => {o, c}
    see => {(b, o), (c, b), (o, c)}
"""
val = nltk.Valuation.fromstring(v)
print(val)

In [ ]:
('o', 'c') in val['see']

In [ ]:
('b',) in val['boy']

## 3.5   Individual Variables and Assignments

In [ ]:
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')])

In [ ]:
print(g)

In [ ]:
m = nltk.Model(dom, val)
m.evaluate('see(olive, y)', g)

In [ ]:
g['y']

In [ ]:
m.evaluate('see(y, x)', g)

In [ ]:
g

In [ ]:
g.purge()
g

In [ ]:
m.evaluate('see(olive, y)', g)

In [ ]:
m.evaluate('see(bertie, olive) & boy(bertie) & -walk(bertie)', g)

## 3.6   Quantification

In [ ]:
m.evaluate('exists x.(girl(x) & walk(x))', g)

In [ ]:
m.evaluate('girl(x) & walk(x)', g.add('x', 'o'))

In [12]:
fmla1 = read_expr('girl(x) | boy(x)')
m.satisfiers(fmla1, 'x', g)

NameError: name 'm' is not defined

In [13]:
fmla2 = read_expr('girl(x) -> walk(x)')
m.satisfiers(fmla2, 'x', g)

NameError: name 'm' is not defined

In [14]:
fmla3 = read_expr('walk(x) -> girl(x)')
m.satisfiers(fmla3, 'x', g)

NameError: name 'm' is not defined

In [15]:
m.evaluate('all x.(girl(x) -> walk(x))', g)

NameError: name 'm' is not defined

In [16]:
m.evaluate('all x.(girl(x) & walk(x))', g)

NameError: name 'm' is not defined

In [17]:
m.evaluate('exists x.(boy(x) -> walk(x))', g)

NameError: name 'm' is not defined

## 3.7   Quantifier Scope Ambiguity

In [18]:
v2 = """
    bruce => b
    elspeth => e
    julia => j
    matthew => m
    person => {b, e, j, m}
    admire => {(j, b), (b, b), (m, e), (e, m)}
"""
val2 = nltk.Valuation.fromstring(v2)

In [19]:
dom2 = val2.domain
m2 = nltk.Model(dom2, val2)
g2 = nltk.Assignment(dom2)
fmla4 = read_expr('(person(x) -> exists y.(person(y) & admire(x, y)))')
m2.satisfiers(fmla4, 'x', g2)

{'b', 'e', 'j', 'm'}

In [20]:
fmla5 = read_expr('(person(y) & all x.(person(x) -> admire(x, y)))')
m2.satisfiers(fmla5, 'y', g2)

set()

In [21]:
fmla6 = read_expr('(person(y) & all x.((x = bruce | x = julia) -> admire(x, y)))')
m2.satisfiers(fmla6, 'y', g2)

{'b'}

## 3.8   Model Building

In [22]:
a3 = read_expr('exists x.(man(x) & walks(x))')
c1 = read_expr('mortal(socrates)')
c2 = read_expr('-mortal(socrates)')
mb = nltk.Mace(5)
print(mb.build_model(None, [a3, c1]))

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [23]:
print(mb.build_model(None, [a3, c2]))

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [24]:
print(mb.build_model(None, [c1, c2]))

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [25]:
a4 = read_expr('exists y. (woman(y) & all x. (man(x) -> love(x,y)))')
a5 = read_expr('man(adam)')
a6 = read_expr('woman(eve)')
g = read_expr('love(adam,eve)')
mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6])
mc.build_model()

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [26]:
print(mc.valuation)

LookupError: You have to call build_model() first to get a model!

In [27]:
a7 = read_expr('all x. (man(x) -> -woman(x))')
g = read_expr('love(adam,eve)')
mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6, a7])
mc.build_model()

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [28]:
print(mc.valuation)

LookupError: You have to call build_model() first to get a model!

# 4   The Semantics of English Sentences

## 4.1   Compositional Semantics in Feature-Based Grammar

## 4.2   The λ-Calculus

In [29]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr(r'\x.(walk(x) & chew_gum(x))')
expr

<LambdaExpression \x.(walk(x) & chew_gum(x))>

In [30]:
expr.free()

set()

In [31]:
print(read_expr(r'\x.(walk(x) & chew_gum(y))'))

\x.(walk(x) & chew_gum(y))


In [32]:
expr = read_expr(r'\x.(walk(x) & chew_gum(x))(gerald)')
print(expr)

\x.(walk(x) & chew_gum(x))(gerald)


In [33]:
print(expr.simplify())

(walk(gerald) & chew_gum(gerald))


In [34]:
print(read_expr(r'\x.\y.(dog(x) & own(y, x))(cyril)').simplify())

\y.(dog(cyril) & own(y,cyril))


In [35]:
print(read_expr(r'\x y.(dog(x) & own(y, x))(cyril, angus)').simplify())

(dog(cyril) & own(angus,cyril))


In [36]:
expr1 = read_expr('exists x.P(x)')
print(expr1)

exists x.P(x)


In [37]:
expr2 = expr1.alpha_convert(nltk.sem.Variable('z'))
print(expr2)

exists z.P(z)


In [38]:
expr1 == expr2

True

In [39]:
expr3 = read_expr('\P.(exists x.P(x))(\y.see(y, x))')
print(expr3)

(\P.exists x.P(x))(\y.see(y,x))


In [40]:
print(expr3.simplify())

exists z1.see(z1,x)


## 4.3   Quantified NPs

## 4.4   Transitive Verbs

In [41]:
read_expr = nltk.sem.Expression.fromstring
tvp = read_expr(r'\X x.X(\y.chase(x,y))')
np = read_expr(r'(\P.exists x.(dog(x) & P(x)))')
vp = nltk.sem.ApplicationExpression(tvp, np)
print(vp)

(\X x.X(\y.chase(x,y)))(\P.exists x.(dog(x) & P(x)))


In [42]:
print(vp.simplify())

\x.exists z2.(dog(z2) & chase(x,z2))


In [43]:
from nltk import load_parser
parser = load_parser('grammars/book_grammars/simple-sem.fcfg', trace=0)
sentence = 'Angus gives a bone to every dog'
tokens = sentence.split()
for tree in parser.parse(tokens):
    print(tree.label()['SEM'])

all z4.(dog(z4) -> exists z3.(bone(z3) & give(angus,z3,z4)))


In [44]:
sents = ['Irene walks', 'Cyril bites an ankle']
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
for results in nltk.interpret_sents(sents, grammar_file):
    for (synrep, semrep) in results:
        print(synrep)

(S[SEM=<walk(irene)>]
  (NP[-LOC, NUM='sg', SEM=<\P.P(irene)>]
    (PropN[-LOC, NUM='sg', SEM=<\P.P(irene)>] Irene))
  (VP[NUM='sg', SEM=<\x.walk(x)>]
    (IV[NUM='sg', SEM=<\x.walk(x)>, TNS='pres'] walks)))
(S[SEM=<exists z5.(ankle(z5) & bite(cyril,z5))>]
  (NP[-LOC, NUM='sg', SEM=<\P.P(cyril)>]
    (PropN[-LOC, NUM='sg', SEM=<\P.P(cyril)>] Cyril))
  (VP[NUM='sg', SEM=<\x.exists z5.(ankle(z5) & bite(x,z5))>]
    (TV[NUM='sg', SEM=<\X x.X(\y.bite(x,y))>, TNS='pres'] bites)
    (NP[NUM='sg', SEM=<\Q.exists x.(ankle(x) & Q(x))>]
      (Det[NUM='sg', SEM=<\P Q.exists x.(P(x) & Q(x))>] an)
      (Nom[NUM='sg', SEM=<\x.ankle(x)>]
        (N[NUM='sg', SEM=<\x.ankle(x)>] ankle)))))


In [45]:
v = """
... bertie => b
... olive => o
... cyril => c
... boy => {b}
... girl => {o}
... dog => {c}
... walk => {o, c}
... see => {(b, o), (c, b), (o, c)}
... """
val = nltk.Valuation.fromstring(v)
g = nltk.Assignment(val.domain)
m = nltk.Model(val.domain, val)
sent = 'Cyril sees every boy'
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
results = nltk.evaluate_sents([sent], grammar_file, m, g)[0]
for (syntree, semrep, value) in results:
    print(semrep)
    print(value)

all z6.(boy(z6) -> see(cyril,z6))
True


## 4.5   Quantifier Ambiguity Revisited

In [46]:
from nltk.sem import cooper_storage as cs
sentence = 'every girl chases a dog'
rees = cs.parse_with_bindops(sentence, grammar='grammars/book_grammars/storage.fcfg')
semrep = trees[0].label()['SEM']
cs_semrep = cs.CooperStore(semrep)
print(cs_semrep.core)

TypeError: tuple indices must be integers or slices, not str

In [47]:
for bo in cs_semrep.store:
    print(bo)

NameError: name 'cs_semrep' is not defined

In [48]:
cs_semrep.s_retrieve(trace=True)

NameError: name 'cs_semrep' is not defined

In [49]:
for reading in cs_semrep.readings:
    print(reading)

NameError: name 'cs_semrep' is not defined

# 5   Discourse Semantics

## 5.1   Discourse Representation Theory

In [51]:
read_dexpr = nltk.sem.DrtExpression.fromstring
drs1 = read_dexpr('([x, y], [angus(x), dog(y), own(x, y)])')
print(drs1)

([x,y],[angus(x), dog(y), own(x,y)])


In [52]:
drs1.draw()

In [53]:
print(drs1.fol())

exists x y.(angus(x) & dog(y) & own(x,y))


In [54]:
drs2 = read_dexpr('([x], [walk(x)]) + ([y], [run(y)])')
print(drs2)

(([x],[walk(x)]) + ([y],[run(y)]))


In [55]:
print(drs2.simplify())

([x,y],[walk(x), run(y)])


In [56]:
drs3 = read_dexpr('([], [(([x], [dog(x)]) -> ([y],[ankle(y), bite(x, y)]))])')
print(drs3.fol())

all x.(dog(x) -> exists y.(ankle(y) & bite(x,y)))


In [57]:
drs4 = read_dexpr('([x, y], [angus(x), dog(y), own(x, y)])')
drs5 = read_dexpr('([u, z], [PRO(u), irene(z), bite(u, z)])')
drs6 = drs4 + drs5
print(drs6.simplify())

([u,x,y,z],[angus(x), dog(y), own(x,y), PRO(u), irene(z), bite(u,z)])


In [58]:
print(drs6.simplify().resolve_anaphora())

([u,x,y,z],[angus(x), dog(y), own(x,y), (u = [x,y,z]), irene(z), bite(u,z)])


In [59]:
from nltk import load_parser
parser = load_parser('grammars/book_grammars/drt.fcfg', logic_parser=nltk.sem.drt.DrtParser())
trees = list(parser.parse('Angus owns a dog'.split()))
print(trees[0].label()['SEM'].simplify())

([x,z11],[Angus(x), dog(z11), own(x,z11)])


## 5.2   Discourse Processing

In [60]:
dt = nltk.DiscourseTester(['A student dances', 'Every student is a person'])
dt.readings()

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [61]:
dt.add_sentence('No person dances', consistchk=True)

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [62]:
dt.retract_sentence('No person dances', verbose=True)

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [63]:
dt.add_sentence('A person dances', informchk=True)

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [64]:
from nltk.tag import RegexpTagger
tagger = RegexpTagger(
    [('^(chases|runs)$', 'VB'),
     ('^(a)$', 'ex_quant'),
     ('^(every)$', 'univ_quant'),
     ('^(dog|boy)$', 'NN'),
     ('^(He)$', 'PRP')
])
rc = nltk.DrtGlueReadingCommand(depparser=nltk.MaltParser(tagger=tagger))
dt = nltk.DiscourseTester(['Every dog chases a boy', 'He runs'], rc)
dt.readings()

TypeError: __init__() missing 1 required positional argument: 'parser_dirname'

In [65]:
dt.readings(show_thread_readings=True)

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [66]:
dt.readings(show_thread_readings=True, filter=True)

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================